In [49]:
import openai
import minsearch
import json
import os
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests

# Initialize OpenAI client
#client = OpenAI(
    # Replace with your OpenAI API key#
#    api_key=os.environ.get("OPENAI_API_KEY"),)

# Initialize Elasticsearch client
es_client = Elasticsearch('http://localhost:9200')

# Initialize minsearch Index
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)


In [50]:
#documents = []
len(documents)

948

In [51]:
with open('documents.json', 'rt') as f:
    docs_raw = json.load(f)

# Flatten and preprocess documents
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
es_client.indices.create(index=index_name,body =index_settings)

In [48]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document =doc)

  0%|                                                                | 0/948 [00:00<?, ?it/s]


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x78203c388bb0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [6]:
index.fit(documents)


In [9]:
def search_query(query):
    boost = {'question': 3.0, 'section': 0.5}
    result = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return result


def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [11]:
def elastic_search_query(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index="course-questions", body=search_query)
    
    results_doc = [hit['_source'] for hit in response['hits']['hits']]
    return results_doc


In [12]:
query2 =  "How do I execute a command in a running docker container?"

In [13]:
result = elastic_search_query(query2)

/tmp/ipykernel_18365/3042410418.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es_client.search(index="course-questions", body=search_query)


In [14]:
len(build_prompt(query2,result))

1513

In [15]:
result


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

In [73]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 870.4 kB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 16.5 MB/s eta 0:00:00m eta 0:00:01


In [75]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [83]:
len(encoding.encode(str(result)))

352

In [ ]:
query = "how do I run kafka?"

def rag(query):
    search_results = search_query(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

rag(query)


In [34]:
url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json"
response = requests.get(url)

if response.status_code == 200:
    with open("documents.json", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


File downloaded successfully.
